In [25]:
!pip install wandb

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import transforms
import wandb
from tqdm import tqdm

In [27]:
def calc_acc(preds,labels):
  _,preds_max = torch.max(preds, 1)
  acc = torch.sum(preds_max==labels.data,dtype = torch.float64)/len(preds)
  return acc

In [35]:
class MyModel(nn.Module):
      def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 128 , (3,3), (1,1), (1,1))
        self.conv2 = nn.Conv2d(128, 64 , (3,3), (1,1), (1,1))
        self.conv3 = nn.Conv2d(64, 64 , (3,3), (1,1), (1,1))
        self.conv4 = nn.Conv2d(64, 32 , (3,3), (1,1), (1,1))

        self.f1 = nn.Linear(32*8*8,512)
        self.f2 = nn.Linear(512,10)

      def forward(self,x):
        x =F.relu( self.conv1(x))
        # x = F.relu(x)
        # x = F.max_pool2d(x, kernel_size=(2,2))
        x = F.relu(self.conv2(x))
        # x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=(2,2))

        x = F.relu(self.conv3(x))
        # x = F.relu(x)
        # x = F.max_pool2d(x, kernel_size=(2,2))
        x = F.relu(self.conv4(x))

        x = F.max_pool2d(x, kernel_size=(2,2))
        # x = nn.MaxPool2d(x)
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.f1(x))
        x = F.relu(self.f2(x))

        x = torch.softmax(x ,dim = 1)
        print(x.shape)

        return x



In [36]:
# device =torch.device("cpu")
# args.gpu
# device = torch.device("cuda" is torch.cuda.is_available  and args.gpu else "cpu")
device = torch.device("cuda" if torch.cuda.is_available()  else  "cpu")
model = MyModel().to(device)
# model = model.to(device)
model.train(True)

MyModel(
  (conv1): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (f1): Linear(in_features=2048, out_features=512, bias=True)
  (f2): Linear(in_features=512, out_features=10, bias=True)
)

In [37]:
# Hyper parameter
batch_size=64
epoch=20
config = wandb.config
config.learning_rate = 0.001

In [38]:
transform = transforms.Compose([
                                transforms.RandomRotation(10),
                                torchvision.transforms.transforms.Resize((70,70)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

dataset = torchvision.datasets.ImageFolder(root = "/content/drive/MyDrive/MNIST_persian",transform = transform )
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [39]:
#!rm -rf dataset # command for remove a folder in colab
#compile
optimizer = torch.optim.RMSprop(model.parameters(),lr=config.learning_rate)
loss_function = nn.CrossEntropyLoss()

In [40]:
wandb.init(project="persian_mnist")

In [41]:
model.train()
for epochs in range(epoch):
    train_loss = 0.0
    train_acc =0.0
    for images , labels in tqdm(train_data_loader):
      images, labels = images.to(device), labels.to(device)
      optimizer.zero_grad()

      preds =model(images)
      loss = loss_function(preds, labels)
      loss.backward()
      optimizer.step()
      train_acc+= calc_acc(preds, labels)
      train_loss+=loss
    total_loss = train_loss/len(train_data_loader)
    total_acc = train_acc/len(train_data_loader)
    print(f"Epoch:{epochs}: Loss:{total_loss}" )
    print(f"Acc:{epochs}: acc:{total_acc}" )
    print("-------------------------------------")
    wandb.log({'Epoch':  epochs + 1,
                'Loss': total_loss,
                'Acc': total_acc
                                })

  0%|          | 0/19 [00:00<?, ?it/s]


RuntimeError: ignored